# TODO: TITLE

## Introduction

Many people in the world suffer from heart disease. Heart disease usually means the narrowing of blood vessels, which is related to several factors such as age, angina, blood pressure, and serum cholesterol levels. Heart problems are common in older adults due to aging arteries. Coronary artery disease is a heart condition with angina. Blood pressure can reflect the health of blood vessels, while serum cholesterol values can also indicate potential blood flow restrictions in blood vessels. So what is the predictive status of vessel stenosis based on patient age, type of chest pain, resting blood pressure, and serum cholesterol values? This project will use a random dataset Reprocessed Hungarian which contains all the required variables above.

## Preliminary Data Analysis

- Demonstrate dataset can be read into R
- Clean and wrangle data
- Summarize data in at least one table
- Vizualize the data with at least one plot

## Methods
- Explain how we will conduct our data analysis and what variables/columns we will use
    - Age
    - Serum choloesterol
    - Chest pain type
    - Resting blood pressure
- Describe at least one way we will visualize results

## Expected Outcomes
- What do we expect to find?
- What impact could such findings have?
- What future questions could this lead to?